## VRP SHEDULING

In [64]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
import numpy as np
import scipy as sp
from numpy import matrix

## Input data

In [65]:
## TEST DATA - SMALL USE CASE

# load dispatsh matrix - demand
    #LJ, CE, MB
D = [17, 6, 25]

# vehicle load capacity
C = [100, 100]

#network graph, stopci = povezave 12, 23, 13, vrstice = mesta
    #LJ,CE,MB
E = [
    [10, 0, 11],
    [0, 21, 26],
    [31, 32, 0],]

#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]

#number of nodes
N = np.size(E,0)
#number of edges
M = np.size(E,1)
#number of vehicles
V = np.size(C)

E

[[10, 0, 11], [0, 21, 26], [31, 32, 0]]

In [474]:

# load dispatsh matrix - demand
    #LJ, CE, MB, KP, NM
D = [17, 6, 25, 37, 12]

# vehicle load capacity
C = [100, 100, 100]

#network graph, stopci = povezave 12, 23, 34, 45, 35, 13, vrstice = mesta
    #LJ,CE,MB,KP,NM
E = [
    [10, 0, 0, 0, 16, 11],
    [0, 21, 23, 0, 0, 26],
    [31, 32, 0, 0, 0, 0],
    [0, 0, 0, 44, 45, 0],
    [0, 0, 53, 54, 0, 0]]

#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]
#-E (negative values)
negE=np.negative(E)

#number of nodes
N = np.size(E,0)
#number of edges
M = np.size(E,1)
#number of vehicles
V = np.size(C)

#[0, 0, 1, 1, 0, 0]
E

[[10, 0, 0, 0, 16, 11],
 [0, 21, 23, 0, 0, 26],
 [31, 32, 0, 0, 0, 0],
 [0, 0, 0, 44, 45, 0],
 [0, 0, 53, 54, 0, 0]]

## Creating Constraint matrix A

In [66]:
#creating A1 matrix - constraint IV, graph data 
#number of rows = number of columns (E) x number of cycles * 2 (for negative and positive notation);
#columns = [number of columns (E) x number of cycles] + [number of rows (E) x number of cycles]
# create zero A1 matrix
#len(Et) = number ob edges
#len(E) = number of nodes
# V = number of cycles

A1 = np.zeros((len(Et)*V*2, len(E)*V+len(Et)*V*2))
for j in range (0, len(Et)):
    for k in range (0, V):
        for i in range (0, len(E)):
            A1[j*V+k][k*len(E)+i]=Et[j][i]
            #print (j*V+k, k*len(E)+i, j, i) #=np.negative(Et[j][i])
for j in range (0, len(Et)):
    for k in range (0, V):
        for i in range (0, len(E)):
            A1[len(Et)*V+j*V+k][k*len(E)+i]=np.negative(Et[j][i])
            #print ( len(Et)*V+ j*V+k, k*len(E)+i, j, i) #=np.negative(Et[j][i])
            
# Adding coeficients for slack variables 
for j in range (0, len(Et)*V): 
    A1[j][len(E)*V+j] = -2
for j in range (0, len(Et)*V): 
    A1 [len(Et)*V+j][len(Et)*V+len(E)*V+j] = 2

In [67]:
# creating matrix A2

    #number of rows: 2 * num. of cycles * num. of nodes (constraint II) + num of cycles (constraint III)
    #number of columns: 2 * num. of cycles * num. of nodes
    
#II constraint, the number of packets delivered on the node is equal to all total demand on the node
size= (2*len(E)+ (V*len(E)))
A2 = np.zeros((size, (len(E)*V)))
       
for j in range (0, len(E)):            #adding constraints load on nodes
    for i in range (j*V, (j+1)*V):
        #print("value is", j, i)
        A2[j,i]=1
for j in range (0, len(E)):
    for i in range (j*V, (j+1)*V):
        #print("value is", j+V, i)
        A2[len(E)+j,i]=-1       
for j in range (0, V*len(E)):         #adding constraints for slack variables
        #print("value is", 2*V+j, j)
        A2[2*len(E)+j,j]=-1

In [68]:
#ADDING CONSTRAINT III to A2 matrix: 
A3 = np.zeros((V, (len(E)*V)))
for j in range (0, V):                                  # for each vehicle
    for i in range (0, len(E)):                         #take the sum of load on enach node
        #print("j", j+i*V)       
        A3[j,j+i*V]=1

for i in range (0, len(A3)):                            #add lines to A2 matrix
    A2=np.vstack([A2, A3[i,:]])

In [69]:
#concatenate A1 and A2 = A matrix
A1extend = np.c_[A1, np.zeros((len(A1), len(A2[0])))]
A2extend = np.c_[np.zeros((len(A2), len(A1[0]))), A2]

A=A1extend.copy()
for i in range (0, len(A2)):
    A=np.vstack([A, A2extend[i,:]])

## Creating & declaring variables

In [70]:
# CREATE VARIABLES  X - vector with c11-cnn variables
# vsak vektor v matriki X1n = dolžine len(E). 
#Skupno število vseh vektorjev X1n = len (Et)
#X variables X[0] to X[len(E) * V -1] -  variables in objective function 
#K - X[len(E) * V] to X[len(E) * V + len(A1) -1]  slack variables
#Ow - cycles load dispatch variables - X[len(E) * V + len(Et)*V*2] to X[len(E) * V + len(Et)*V*3 -1 ] 

X1size = len(E) * V   # number of all variables
X1 = []
for i in range (0, X1size):
    var='x'+str(i)
    X1.append(var)
 
#creating vector with K variables
K = []
for j in range (0, len(A1)):
    #creating vector with K variables
    var='k'+str(j)
    K.append(var)
    
#creating vector with "O" variables
Ow = []
for j in range(0, V*len(Et)):
    var='Ow'+str(j)
    Ow.append(var)
    
#Final X vector with all variables
X=X1.copy()
for i in range (0,len(K)):
    X.append(K[i])
    #print (K[i])

for j in range (0,len(Ow)):
    X.append(Ow[j])
print("number of variables =", len(X))

number of variables = 24


In [71]:
## Declaring variables to the solver

#Declaring the solver
solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x_min = 0.0                                # lower variables border 
x_max = solver.infinity()                  # Upper variables border                        
#X_int=X1
#X_real=K
#def declare_variables(X_int, X_real,x_min, x_max):

variables = [] 

for varN, xi_name in enumerate(X1):                          # declaring objective variables 
    variables.append(solver.IntVar(x_min, x_max, xi_name))
    
for varN, xi_name in enumerate(K):                          # declaring slack variables
    variables.append(solver.NumVar(x_min, x_max, xi_name))

for varN, xi_name in enumerate(Ow):                          # declaring load doispatch variables
    variables.append(solver.IntVar(x_min, x_max, xi_name))

print('Number of variables created =', solver.NumVariables())
    #for variable in variables:
        #print('%s = %d' % (variable.name(), variable.solution_value()))

Number of variables created = 24


## Declare constraints

In [74]:
#EQUATIONS COEFICIENTS 

#coeficients for constraint I
b1=[]
for i in range (0, len(A1)):
    b1.append(0)

# coeficients for A2 matrix: b2+b3
b21=[]
b21 = D + [-val for val in D] 
b22=[]
for i in range (0, V*len(E)):
    b22.append(0)
#coeficients for A3 constraint III ; b3 = C, vector of vehicle capacities
if len(b21+b22+C) != len(A2):
    print("Error on A2 constants")
b3=C

# concatenated vector of constants - b    
b= b1 + b21 + b22 + b3


In [75]:
#DECLARE CONSTRAINTS
#b = np.zeros((1, len(A)))

for rowN, row in enumerate(A):
    left_side = None
    for colN, coeff in enumerate(row):
        if coeff == 0:
            continue
        if left_side is None:
            left_side = coeff*variables[colN]
        else:
            left_side += coeff*variables[colN]
    if left_side is None and b[0,rowN] < 0:
        raise ValueError('Constraint ' + str(rowN) + ' cannot be satisfied!')
    if left_side is not None:
        #print (left_side)
       # solver.Add(left_side <= 0)
       #solver.Add(left_side <= t[0,rowN])
        solver.Add(left_side <= b[rowN])
        #print (b[rowN])
        
print('Number of constraints added =', solver.NumConstraints())

Number of constraints added = 26


## Declare Objective Function

In [81]:
# declare objective function: x0+x1+x2+x3+x4+......

#C1 = [variables[0], variables[1], variables[2], variables[3], variables[4], variables[5]]
C = []
for i in range (0, len(X1)-1):
    C.append(variables[i])
    #print (variables[i])
objective = sum (C)
solver.Minimize(objective)

# Weights for prioritizing different variables
#wgt_X1 = 1.0
#X1_coeffs = [ wgt_X1 for _ in X1 ]
#for coeffN, coeff in enumerate(X1_coeffs):
#    if coeff != 0:
#        C += coeff*X1[coeffN] 

#solver.Maximize(x0 + x1 + x2 + x3 + x4 + x5 + x6 +x7 + x8 + x9)   

## Invoke the solver

In [84]:
result_status = solver.Solve()
    # The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL
result_status

0

In [85]:
for i in range (0, len(X)):
    print('variable', i, variables[i].solution_value())

variable 0 0.0
variable 1 0.0
variable 2 0.0
variable 3 0.0
variable 4 0.0
variable 5 0.0
variable 6 0.0
variable 7 0.0
variable 8 0.0
variable 9 0.0
variable 10 0.0
variable 11 0.0
variable 12 0.0
variable 13 0.0
variable 14 0.0
variable 15 0.0
variable 16 0.0
variable 17 0.0
variable 18 17.0
variable 19 0.0
variable 20 6.0
variable 21 0.0
variable 22 25.0
variable 23 0.0


In [86]:
assert solver.VerifySolution(1e-7, True)

print('Solution:')
print('Objective value =', solver.Objective().Value())
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())

print('x0 =', variables[0].solution_value())
print('x1 =', variables[1].solution_value())

Solution:
Objective value = 0.0
Number of variables = 24
Number of constraints = 26

Advanced usage:
Problem solved in 239870.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes
x0 = 0.0
x1 = 0.0
